In [1]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [2]:
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
client = Client(n_workers=12, memory_limit='16GB')

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 178.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61712,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 178.81 GiB
Comm: tcp://127.0.0.1:61763,Total threads: 1
Dashboard: http://127.0.0.1:61773/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:61715,


In [5]:
df = dd.read_csv(r'D:\00 Datasets\CICAPT-IIoT Dataset\Network_Traffic\Phase 1\phase1_NetworkData.csv', blocksize = '80MB')
df

,ts,flow_duration,Header_Length,Source IP,Destination IP,Source Port,Destination Port,Protocol Type,Protocol_name,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,urg_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,max_duration,min_duration,sum_duration,average_duration,std_duration,CoAP,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,MAC,Magnitue,Radius,Covariance,Variance,Weight,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,label,subLabel,subLabelCat
npartitions=68,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,int64,string,string,int64,int64,int64,string,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,int64,float64,float64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
# Compute the number of rows
num_rows = df.shape[0].compute()
# Print the number of rows
print("Number of rows:", num_rows)

# Compute the number of columns
num_columns = len(df.columns)

# Print the number of columns
print("Number of columns:", num_columns)

Number of rows: 12062396
Number of columns: 70


In [7]:
# To print the column names
print(df.columns)

Index(['ts', 'flow_duration', 'Header_Length', 'Source IP', 'Destination IP',
       'Source Port', 'Destination Port', 'Protocol Type', 'Protocol_name',
       'Duration', 'Rate', 'Srate', 'Drate', 'fin_flag_number',
       'syn_flag_number', 'rst_flag_number', 'psh_flag_number',
       'ack_flag_number', 'urg_flag_number', 'ece_flag_number',
       'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count',
       'rst_count', 'max_duration', 'min_duration', 'sum_duration',
       'average_duration', 'std_duration', 'CoAP', 'HTTP', 'HTTPS', 'DNS',
       'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP',
       'IGMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size',
       'IAT', 'Number', 'MAC', 'Magnitue', 'Radius', 'Covariance', 'Variance',
       'Weight', 'DS status', 'Fragments', 'Sequence number',
       'Protocol Version', 'flow_idle_time', 'flow_active_time', 'label',
       'subLabel', 'subLabelCat'],
      dtype='object')


In [ ]:
df.head()  # conda install -c conda-forge msgpack-python==1.0.5

In [8]:
df.isna().sum()

Dask Series Structure:
npartitions=1
ARP                int64
urg_flag_number      ...
Dask Name: sum, 3 expressions
Expr=IsNa(frame=ReadCSV(22da3c8)).sum()

In [9]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns.tolist()

print("Columns with non-numeric values:")
print(non_numeric_columns)

Columns with non-numeric values:
['Source IP', 'Destination IP', 'Protocol_name']


In [ ]:
# Remove leading and trailing spaces from column names
df.columns = df.columns.str.strip()  

In [ ]:
Source_IP_names = df['Source IP'].unique().compute()

print("Number of Source IP Names:", len(Source_IP_names))
print("Source IP Names:")
print(Source_IP_names)

In [ ]:
Destination_IP_names = df['Destination IP'].unique().compute()

print("Number of Destination IP Names:", len(Destination_IP_names))
print("Source IP Names:")
print(Destination_IP_names)

In [ ]:
unique_protocol_names = df['Protocol_name'].unique().compute()

print("Number of Unique Protocol Names:", len(unique_protocol_names))
print("Unique Protocol Names:")
print(unique_protocol_names)

In [ ]:
label_names = df['label'].unique().compute()

print("Number of label_names:", len(label_names))
print("label_names:")
print(label_names)

In [ ]:
subLabel_names = df['subLabel'].unique().compute() 

print("Number of subLabel_names:", len(subLabel_names))
print("subLabel_names:")
print(subLabel_names)

In [ ]:
subLabelCat_names = df['subLabelCat'].unique().compute()

print("Number of subLabelCat_names:", len(subLabelCat_names))
print("subLabelCat_names:")
print(subLabelCat_names)

In [ ]:
# Step 1: Convert the relevant columns to categorical type
df = df.categorize(columns=['Source IP', 'Destination IP', 'Protocol_name'])

# Step 2: Apply one-hot encoding using Dask's get_dummies
df_encoded = dd.get_dummies(df, columns=['Source IP', 'Destination IP', 'Protocol_name'])

# Step 3: Display the first few rows of the transformed DataFrame
print(df_encoded.head())

In [ ]:
# To display the first few rows of data
df_encoded.head()  # By default, it shows the first 5 rows

In [ ]:
# Save the filtered DataFrame to a CSV file
# df_encoded.to_csv('df_encoded/*.csv', index=False, single_file=True)  # Save as a single CSV file
df_encoded.to_csv('df_encoded.csv', index=False, single_file=True)

print("Encoded Columns saved as df_encoded.csv'.")

In [ ]:
mean_values = df_encoded.mean()  # Returns a series
print(mean_values)  

In [ ]:
df_encoded.head()

num_rows = df_encoded.shape[0]
num_cols = df_encoded.shape[1]

print("Number of rows in the sample:", num_rows)
print("Number of columns in the sample:", num_cols)

In [ ]:
df_encoded.describe()

In [ ]:
df_encoded = dd.read_csv(r'D:\df_encoded.csv', blocksize = '80MB')
df_encoded

In [ ]:
# Select a random sample or downsample the DataFrame
df_encoded_sampled = df_encoded.sample(frac=0.1, random_state=42)
df_encoded_sampled.to_csv('df_encoded_sampled.csv', index=False, single_file=True)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df_encoded_sampled)
df_pca_all = pca.transform(df_encoded_sampled)
eigenvalues = pca.explained_variance_
plt.bar(np.arange(0,df_encoded_sampled.shape[1],1), eigenvalues)
plt.plot(eigenvalues, "r")
plt.plot(eigenvalues, "ro")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df_encoded)
df_pca_all = pca.transform(df_encoded)
eigenvalues = pca.explained_variance_
plt.bar(np.arange(0,df_encoded.shape[1],1), eigenvalues)
plt.plot(eigenvalues, "r")
plt.plot(eigenvalues, "ro")
plt.show()